In [4]:
## Description: This file contains the utility functions for the DubLingo application.
import json
import wave
import os
import numpy as np
import requests
import base64
import io
import re
from pydub import AudioSegment

ModuleNotFoundError: No module named 'requests'

In [1]:
# Function to get speaker-wise audio
def get_speaker_wise_audio(audio_file,json_file,output_dir):

    def time_in_milliSecond(time_str):
        hours, minutes, seconds_milliseconds = time_str.split(":")
        seconds, milliseconds = seconds_milliseconds.split(",")
        milliseconds=milliseconds[0]+milliseconds[1]
        total_milliseconds = int(hours) * 60 * 60 * 1000 + int(minutes) * 60 * 1000 + int(seconds) * 1000 + int(milliseconds)
        return total_milliseconds

    def cut_wav_file(input_file, output_file, max_duration=20000):
        # Load the audio file
        audio = AudioSegment.from_wav(input_file)

        # Check the duration of the audio
        duration = len(audio)

        # If the duration is longer than max_duration, cut the audio to 20 seconds
        if duration > max_duration:
            audio = audio[:max_duration]

        # Export the modified audio to a new file
        audio.export(output_file, format="wav")


    def split_audio_by_speaker(audio_file, json_file):
        """Splits an audio file into speaker-wise chunks based on a JSON file.

        Args:
            audio_file (str): Path to the audio file to split.
            json_file (str): Path to the JSON file containing speaker information.

        Returns:
            list: A list of speaker-wise audio chunks.
        """

        # Load the JSON file
        with open(json_file,encoding='utf8') as f:
            data = json.load(f)

        # Create a dictionary to store the audio chunks
        chunks = {}


        # Open the audio file
        wav = wave.open(audio_file, 'rb')
        # Iterate over the JSON data
        for entry in data:
            # Get the speaker ID and start/end times
            speaker_id = entry["speaker"]

            # # Read in MilliSeoncds
            frame_rate = wav.getframerate()

            # Convert start and end times from milliseconds to frames
            start_time_ms = int(time_in_milliSecond(entry["startTime"]))  # Example: start time in milliseconds
            end_time_ms = int(time_in_milliSecond(entry["endTime"]))    # Example: end time in milliseconds

            start_frame = int(start_time_ms / 1000 * frame_rate)
            end_frame = int(end_time_ms / 1000 * frame_rate)

            # Move the file pointer to the start frame
            wav.setpos(start_frame)

            # Read frames between start_frame and end_frame
            frames = wav.readframes(end_frame - start_frame)


            # Add the audio chunk to the dictionary
            if speaker_id not in chunks:
                chunks[speaker_id] = []
            chunks[speaker_id].append(frames)

        # Close the audio file
        wav.close()
        # Return the list of speaker-wise audio chunks
        return chunks


    chunks = split_audio_by_speaker(audio_file, json_file)

    # Print the speaker-wise audio chunks
    for speaker_id, frames in chunks.items():
        combined_frames = b"".join([chunk for chunk in frames])

        # Create the output file with speaker ID
        output = f"{output_dir}{speaker_id}.wav"

        # Open the original audio file to get parameters
        with wave.open(audio_file, "rb") as original_wav:
            params = original_wav.getparams()

        # Write the combined frames to the output file
        with wave.open(output, "wb") as out_wav:
            out_wav.setparams(params)
            out_wav.writeframes(combined_frames)
        
        # cut the audio to 20 seconds  
        cut_wav_file(output, output,max_duration=20000)

# Replace this with the actual path to your video file
video_path = output_dir+'video.mp4'
target_json_filename='arabicTranslation.json'
source_json_filename='urduTranscription.json'
source_wav_vocals_filename='vocals.wav'
source_wav_music_filename='music.wav'
output_video_path=output_dir+'arabicVideo.mp4'
get_speaker_wise_audio(output_dir+source_wav_vocals_filename,output_dir+target_json_filename,output_dir):

SyntaxError: invalid syntax (1129818730.py, line 108)

In [3]:
# Function to generate and save audio
def generate_and_save_audio(json_file,output_dir):
    def generate_audio(text, speaker_name, sentence_id,output_dir):
        # Replace this with the actual URL of the TTS service
        input_audio_file=f"{output_dir}{speaker_name}.wav"
        audio_path = f"{output_dir}{speaker_name}_{sentence_id}.wav"
        print(input_audio_file,audio_path)
        # Generate speech using the saved audio file and provided or default text
        tts.tts_to_file(text=text,
                        file_path=audio_path,
                        speaker_wav=input_audio_file,  # Use the saved path
                        language="ar",
                        split_sentences=True,
                        speed=0.4)


    with open(json_file, 'r', encoding='utf8') as json_file:
        data = json.load(json_file)
    speakers=[]
    for entry in data:
        speaker_name_current = entry["speaker"]
        sentence_id = entry["sentence_id"]
        text = entry["translation"]
        if speaker_name not in speakers:
            for data_entry in data:
                speaker_name = data_entry["speaker"]
                if speaker_name == speaker_name_current and speaker_name not in speakers:
                    sentence_id = entry["sentence_id"]
                    text = entry["translation"]
                    generate_audio(text, speaker_name, sentence_id,output_dir)
                else:
                    sentence_id = entry["sentence_id"]
                    text = entry["translation"]
                    generate_audio(text, speaker_name, sentence_id,output_dir)
            speakers.append(speaker_name)

In [ ]:
def generate_clone_voices():
    output_dir='./content/'
    target_json_filename='arabicTranslation.json'
    source_auido_filename='vocals.wav'
    audio_file = output_dir+'output_directory/Test/'+source_auido_filename
    json_file = output_dir+target_json_filename

    # get data from requets and save it to the target_json_filename
    # check that response in correct format
    data=response.json()
    if not os.path.exists(json_file):
        # create the file and then save

    # Check if the file exists
    if not os.path.exists(json_file):
        # If the file doesn't exist, create it and write an empty JSON object
        with open(json_file, 'w') as f:
            json.dump({}, f)
    with open(json_file, 'w', encoding='utf8') as json_file:
        json.dump(data, json_file, ensure_ascii=False)

    get_speaker_wise_audio(audio_file,json_file,output_dir)
    generate_and_save_audio(json_file,output_dir)

    # Load json file
    with open(json_file, 'r', encoding='utf8') as json_file:
        data = json.load(json_file)
        # make response to send back
        response = {}
        for entry in data:
            speaker_name = entry["speaker"]
            sentence_id = entry["sentence_id"]
            audio_path = f"{output_dir}{speaker_name}_{sentence_id}.wav"
            with open(audio_path, 'rb') as f:
                audio_data = f.read()

            audio_b64 = base64.b64encode(audio_data).decode('utf-8')
            response[speaker_name] = audio_b64

    return json.dumps(response)

In [ ]:
def generated_voices(url,output_dir,target_json_filename):
    # make a request to the TTS service by sending the json file and get the audio files
    with open(output_dir+target_json_filename, 'r', encoding='utf8') as json_file:
        data = json.load(json_file)
        succeed=False
        while not succeed:
            try:
                response = requests.post(url+'CloneVoices', files=files, data=data, timeout=100000)
                print(response)
                data = response.json()

                for entry in data:
                    speaker_name = entry["speaker"]
                    sentence_id = entry["sentence_id"]
                    audio_path = f"{output_dir}{speaker_name}_{sentence_id}.wav"
                    audio_name=f"{speaker_name}_{sentence_id}.wav"
                    # Save the received video file
                    audio_b64 = data[audio_name]

                    audio_bytes = base64.b64decode(audio_b64)

                    # Convert bytes to AudioSegment
                    audio = AudioSegment.from_file(io.BytesIO(audio_bytes))
                    # Save AudioSegments as WAV files
                    audio_audio.export(audio_file_path, format='wav')
                succeed=response.ok
            except requests.Timeout:
                print('The request timed out.')
            except requests.RequestException as e:
                print(f'An error occurred: {e}')

In [3]:
filename = 'data.json'
import os 
import json   
# Check if the file exists
if not os.path.exists(filename):
    # If the file doesn't exist, create it and write an empty JSON object
    with open(filename, 'w') as f:
        json.dump({}, f)